In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
import os
os.chdir("/content/gdrive/My Drive/Colab Notebooks")

In [5]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
import xgboost as xgb
from xgboost import plot_importance, plot_tree
from sklearn.metrics import mean_squared_error, mean_absolute_error

In [72]:
import lightgbm as lgb
 
import pickle
 
from sklearn.metrics import roc_auc_score

In [19]:
pjme = pd.read_csv('/content/train.csv', index_col=[1], parse_dates=[1])

In [93]:
split_date = '20180501'
pjme_train = pjme.loc[pjme.index <= split_date].copy()
pjme_test = pjme.loc[pjme.index > split_date].copy()

In [21]:
def create_features(df, label=None):
    """
    Creates time series features from datetime index
    """
    df['date'] = df.index
    df['hour'] = df['date'].dt.hour
    df['dayofweek'] = df['date'].dt.dayofweek
    df['quarter'] = df['date'].dt.quarter
    df['month'] = df['date'].dt.month
    df['year'] = df['date'].dt.year
    df['dayofyear'] = df['date'].dt.dayofyear
    df['dayofmonth'] = df['date'].dt.day
    df['weekofyear'] = df['date'].dt.weekofyear
    
    X = df[['hour','dayofweek','quarter','month','year',
           'dayofyear','dayofmonth','weekofyear']]
    if label:
        y = df[label]
        return X, y
    return X

In [107]:
X_train, y_train = create_features(pjme_train, label='speed')
X_test, y_test = create_features(pjme_test, label='speed')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  del sys.path[0]


reg = xgb.XGBRegressor(n_estimators=1000)
reg.fit(X_train, y_train,
        eval_set=[(X_train, y_train), (X_test, y_test)],
        early_stopping_rounds=50,
       verbose=False) # Change verbose to True if you want to see it train

In [111]:
reg_lgb = lgb.LGBMRegressor(n_estimators=1000)
reg_lgb.fit(X_train, y_train,
        eval_set=[(X_train, y_train), (X_test, y_test)],
        early_stopping_rounds=50,
       verbose=False) # Change verbose to True if you want to see it train

LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
              importance_type='split', learning_rate=0.1, max_depth=-1,
              min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
              n_estimators=1000, n_jobs=-1, num_leaves=31, objective=None,
              random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
              subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [122]:
param = {'max_depth': 6, 'num_leaves': 50, 'num_trees':100, 'objective':'regression', 'iterations':1000, 'learning_rate':0.1, 'metric':'mse'}
data_train2 = lgb.Dataset(X_train, y_train, silent=True)
reg2222 = lgb.train(param, data_train2)
y_pred3 = reg2222.predict(X_test2)

/usr/local/lib/python3.6/dist-packages/lightgbm/engine.py:118: UserWarning: Found `num_trees` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


In [123]:
y_pred3

array([47.33995275, 47.16126863, 37.06849818, ..., 50.18796081,
       45.62926293, 48.12463879])

In [38]:
def create_features2(df, label=None):
    """
    Creates time series features from datetime index
    """
    df['date'] = df.index
    df['hour'] = df['date'].dt.hour
    df['dayofweek'] = df['date'].dt.dayofweek
    df['quarter'] = df['date'].dt.quarter
    df['month'] = df['date'].dt.month
    df['year'] = df['date'].dt.year
    df['dayofyear'] = df['date'].dt.dayofyear
    df['dayofmonth'] = df['date'].dt.day
    df['weekofyear'] = df['date'].dt.weekofyear
    
    X = df[['hour','dayofweek','quarter','month','year',
           'dayofyear','dayofmonth','weekofyear']]
    return X

In [26]:
total_test = pd.read_csv('/content/train.csv', index_col=[1], parse_dates=[1])

In [39]:
t_test = create_features2(total_test, label='speed')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  del sys.path[0]


In [44]:
t_test['speed'] = reg.predict(t_test)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [65]:
t_test.to_csv('/content/t_test.csv')

In [50]:
sub = pd.read_csv('/content/sampleSubmission.csv', index_col=[0])

In [66]:
dateparse = lambda dates: pd.datetime.strptime(dates, '%d/%m/%Y %H:%M')
data_test2 = pd.read_csv('/content/test.csv', parse_dates=['date'],index_col='date',date_parser=dateparse)
print(data_test2.head())

                     id
date                   
2018-01-01 02:00:00   0
2018-01-01 05:00:00   1
2018-01-01 07:00:00   2
2018-01-01 08:00:00   3
2018-01-01 10:00:00   4


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime instead.
  """Entry point for launching an IPython kernel.


In [67]:
X_test2 = create_features(data_test2)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  del sys.path[0]


In [85]:
y_pred2 = reg.predict(X_test2)

In [112]:
y_pred2_lgb = reg_lgb.predict(X_test2)

In [113]:
y_pred2_lgb

array([46.58802868, 46.00777136, 33.30162152, ..., 42.29764071,
       39.74925988, 42.23655884])

In [86]:
id = range(len(y_pred2))
ans = pd.DataFrame({'id':id,'speed':y_pred2}).set_index('id')
ans

,speed
id,
0,47.271252
1,48.518734
2,38.080055
3,28.352142
4,36.583839
...,...
3499,14.538576
3500,26.482187
3501,46.743675


In [114]:
ans.to_csv('/content/result_xgb.csv')

In [117]:
id2 = range(len(y_pred2_lgb))
ans2 = pd.DataFrame({'id':id2,'speed':y_pred2_lgb}).set_index('id')
ans2

,speed
id,
0,46.588029
1,46.007771
2,33.301622
3,25.284185
4,28.922327
...,...
3499,16.535343
3500,26.239680
3501,42.297641


In [118]:
ans2.to_csv('/content/result_lgb.csv')

In [124]:
id3 = range(len(y_pred3))
ans3 = pd.DataFrame({'id':id3,'speed':y_pred3}).set_index('id')
ans3.to_csv('/content/result_3.csv')